In [ ]:
import numpy as np
from scipy.optimize import curve_fit


def modeleval(lam: float,
              k: float,
              inttum: float,
              time) -> np.ndarray:
    """
    Tumour-volume model
        F(t) = inttum·k / [ inttum + (k – inttum)·e^{-lam·(t – t0)} ]

    Parameters
    ----------
    lam     : growth / decay rate lambda
    k       : asymptotic volume (carrying capacity)
    inttum  : initial tumour volume at time[0]
    time    : 1-D array-like of time points (days, hours, …)

    Returns
    -------
    numpy.ndarray
        Model-predicted volume at every time point.
    """
    time = np.asarray(time, dtype=float)
    return (inttum * k) / (inttum + (k - inttum) * np.exp(-lam * (time - time[0])))


def rsseval(lam: float,
            k: float,
            inttum: float,
            time,
            tum_vol,
            tum_sd) -> float:
    """
    Weighted residual-sum-of-squares objective

        RSS = ||(tum_vol − F) / tum_sd||_2^2

    Returns
    -------
    float
        Sum of squared SD-weighted residuals – suitable as a
        scalar objective for `scipy.optimize.minimize`, `least_squares`, etc.
    """
    f = modeleval(lam, k, inttum, time)
    sq_res = np.linalg.norm((np.asarray(tum_vol) - f) / np.asarray(tum_sd))
    return sq_res ** 2


def sofneval(inttum: float,
             n: float, #  | np.ndarray
             params: tuple[float, float]) -> float: #  | np.ndarray
    """
    Solution to dS/dN equation:
    
    S(N) = gamma / [ 1 + (gamma − 1)·e^{rho·(N − inttum)} ]

    Parameters
    ----------
    inttum : reference value (e.g. initial tumour volume)
    n      : scalar or array of n values
    params : (rho, gamma) tuple

    Returns
    -------
    float or numpy.ndarray
        S(N) evaluated at each N.
    """
    rho, gam = params
    return gam / (1 + (gam - 1) * np.exp(rho * (np.asarray(n) - inttum)))

# CONTROL GROUP  ---------------------------------------------------
time_C      = np.array([1, 6, 8, 11, 13], dtype=float)
tum_vol_C   = np.array([0.1120, 0.4912, 0.5730, 1.1082, 1.4465])
tum_sd_C    = np.array([0.0880, 0.3200, 0.3760, 0.1360, 0.3520])

surv_vol_C  = np.array([0.0940, 0.4753, 1.0980, 1.5353, 1.7378, 1.8658])
surv_prob_C = np.array([1.0, 1.0, 0.6, 0.4, 0.2, 0.0])

INTTUM_C = 0.094        # fixed initial tumor volume (control)

def s_of_n_control(n, rho, gam):
    # curve_fit passes N (x-data) first, then each free parameter
    return sofneval(INTTUM_C, n, (rho, gam))

p0_C     = (4.162, 1.012)   # initial guess   (rho, gam)
bounds_C = ([0.0, 1.0],      # lower bounds
            [20.0, 1.1])    # upper bounds

popt_C, pcov_C = curve_fit(
        f       = s_of_n_control,
        xdata   = surv_vol_C,
        ydata   = surv_prob_C,
        p0      = p0_C,
        bounds  = bounds_C
)

rho_C, gam_C = popt_C
sN_C  = sofneval(INTTUM_C, surv_vol_C, popt_C)   # model-predicted survival
SNparams_C = popt_C
SNparams_C = [4.162, 1.012] # midpoint fit
print("CONTROL Survival params rho, gamma =", popt_C)
print("Predicted Survival S(N) =", sN_C)

# IRRADIATED GROUP  ------------------------------------------------
time_IR      = np.array([6, 8, 11, 13, 15, 18, 20], dtype=float)
tum_vol_IR   = np.array([0.296, 0.352, 0.552, 0.712, 1.040, 1.208, 1.328])
tum_sd_IR    = np.array([0.176, 0.176, 0.232, 0.304, 0.328, 0.340, 0.360])

surv_vol_IR  = np.array([0.296, 0.3581, 0.552, 0.8488, 1.0983, 1.3242])
surv_prob_IR = np.array([1.0,   1.0,    1.0,  0.6,    0.2,   0.0])

INTTUM_IR = 0.296        # fixed initial tumor volume (IR)

def s_of_n_ir(n, rho, gam):
    return sofneval(INTTUM_IR, n, (rho, gam))

p0_IR     = (5.693, 1.072)              
bounds_IR = ([0.0, 1.0], [20.0, 1.1])   

popt_IR, pcov_IR = curve_fit(
        f      = s_of_n_ir,
        xdata  = surv_vol_IR,
        ydata  = surv_prob_IR,
        p0     = p0_IR,
        bounds = bounds_IR,
        ftol=1e-9, xtol=1e-9, gtol=1e-9
)

rho_IR, gam_IR = popt_IR
sN_IR = sofneval(INTTUM_IR, surv_vol_IR, popt_IR)
SNparams_IR = popt_IR
SNparams_IR = [5.693, 1.072] # midpoint fit
print("\nPOST-IR Survival params rho, gamma =", popt_IR)
print("Predicted Survival S(N) =", sN_IR)

# EXTERNAL CONTROL GROUP  
time_ext = np.array([1, 4, 7, 10, 14, 17, 21], dtype=float)
tum_vol_ext = np.array([0.094, 0.129, 0.172, 0.266, 0.401, 0.581, 0.842])
tum_sd_ext = np.array([0.025, 0.035, 0.04, 0.045, 0.05, 0.1, 0.105])

In [ ]:
import numpy as np, pandas as pd
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares
import matplotlib.pyplot as plt

tum_sd_Cf    = tum_sd_C
# last two days for IR data has 1 surviving mouse (still 20% of pop), adjusted sd to ensure model capture of data
tum_sd_IRf    = np.array([0.176, 0.176, 0.232, 0.304, 0.328, 0.010, 0.010]) 

# ------------------------------------------------------------------
# INPUT YOUR DATA
# ------------------------------------------------------------------
t1, y1_obs, sd1 = time_C,  tum_vol_C,  tum_sd_Cf      # control
t2, y2_obs, sd2 = time_IR, tum_vol_IR, tum_sd_IRf    # IR

# ------------------------------------------------------------------
# ODE definitions  (λ, K, v, b, n0) signature
# ------------------------------------------------------------------
def ode_exponential(t, y, lam, k, v, b, n0):
    return [lam * y[0]]

def ode_logistic(t, y, lam, k, v, b, n0):
    return [lam * y[0] * (1 - y[0] / k)]

def ode_general_logistic(t, y, lam, k, v, b, n0):
    return [lam * y[0] * (1 - (y[0] / k)**v)]

def ode_gompertz(t, y, lam, k, v, b, n0):
    return [lam * y[0] * np.log(k / y[0])]

def ode_bertalanffy(t, y, lam, k, v, b, n0):
    return [lam * y[0]**v - b * y[0]]

models = {
    "Exponential": ode_exponential,
    "Logistic": ode_logistic,
    "General-Logistic": ode_general_logistic,
    "Gompertz": ode_gompertz,
    "von Bertalanffy": ode_bertalanffy,
}

# Which parameter indices are *used* by each model (in [λ, K, v, b, n0])
USED = {
    "Exponential":      [0, 4],
    "Logistic":         [0, 1, 4],
    "General-Logistic": [0, 1, 2, 4],
    "Gompertz":         [0, 1, 4],
    "von Bertalanffy":  [0, 2, 3, 4],
}

# ------------------------------------------------------------------
# Solver (interpolated)
# ------------------------------------------------------------------
def solve_ode(ode_fun, params, t_eval):
    t_eval = np.asarray(t_eval, float).ravel()
    lam, k, v, b, n0 = params
    t_sorted = np.sort(np.unique(t_eval))
    sol = solve_ivp(lambda t, y: ode_fun(t, y, lam, k, v, b, n0),
                    (t_sorted[0], t_sorted[-1]),
                    [n0], dense_output=True, method="RK45")
    return sol.sol(t_eval)[0]

# ------------------------------------------------------------------
# Residuals using *separate* parameters for datasets
# ------------------------------------------------------------------
def weighted_residuals(params, ode_fun,
                       t1, y1, sd1, t2, y2, sd2):
    p1, p2 = params[:5], params[5:]
    r1 = (solve_ode(ode_fun, p1, t1) - y1)  / sd1
    r2 = (solve_ode(ode_fun, p2, t2) - y2)  / sd2
    return np.concatenate((r1, r2))

# ------------------------------------------------------------------
# Initial guesses and bounds
# ------------------------------------------------------------------
M1 = y1_obs.max()*2
M2 = y2_obs.max()*2
spec = dict(
    p0=np.r_[ (0.3, M1, 1.2, 0.1, y1_obs[0]),
              (0.3, M2, 1.2, 0.1, y2_obs[0]) ],
    lb=np.r_[ (0,   0,  0.5, 0, 0),
              (0,   0,  0.5, 0, 0)],
    ub=np.r_[ (2,  10,   2,  2, 1),
              (2,  10,   2,  2, 1)]
)

# ------------------------------------------------------------------
# Fit all models
# ------------------------------------------------------------------
rows = []
for name, ode in models.items():
    res = least_squares(weighted_residuals, spec["p0"],
                        bounds=(spec["lb"], spec["ub"]),
                        args=(ode, t1, y1_obs, sd1, t2, y2_obs, sd2),
                        method="trf")
    theta = res.x
    resid = res.fun
    n_obs = resid.size
    logL = -0.5 * np.sum(resid**2 + np.log(2*np.pi))
    
    # Compute k_eff (only count active parameters per dataset)
    used = USED[name]
    k_eff = 2 * len(used)  # one set per dataset

    AIC = 2 * k_eff - 2 * logL
    BIC = k_eff * np.log(n_obs) - 2 * logL

    rows.append(dict(
        Model=name, AIC=AIC, BIC=BIC, LogLik=logL,
        λ1=theta[0], K1=theta[1], v1=theta[2], b1=theta[3], n01=theta[4],
        λ2=theta[5], K2=theta[6], v2=theta[7], b2=theta[8], n02=theta[9]
    ))

results = pd.DataFrame(rows).set_index("Model").round(3)
print(results.sort_values("AIC"))

fig, axes = plt.subplots(len(models), 1, figsize=(8, 4 * len(models)), sharex=True)

t_fine1 = np.linspace(min(t1), max(t1), 200)
t_fine2 = np.linspace(min(t2), max(t2), 200)

for i, (name, row) in enumerate(results.iterrows()):
    ode_fun = models[name]
    theta = row.filter(regex=r"[λKvbn]")  # gets λ1,K1,...,n02
    theta = theta.values.astype(float)
    p1, p2 = theta[:5], theta[5:]

    yhat1 = solve_ode(ode_fun, p1, t_fine1)
    yhat2 = solve_ode(ode_fun, p2, t_fine2)

    ax = axes[i]
    ax.set_title(f"{name} fit", fontsize=14)

    # Data
    ax.errorbar(t1, y1_obs, yerr=sd1, fmt='o', label='Control (data)', color='#CC5500')
    ax.errorbar(t2, y2_obs, yerr=sd2, fmt='o', label='IR (data)', color='#5B7C99')

    # Fits
    ax.plot(t_fine1, yhat1, '-', label='Control (fit)', color='#CC5500')
    ax.plot(t_fine2, yhat2, '-', label='IR (fit)', color='#5B7C99')

    ax.set_ylabel("Tumor Volume")
    ax.legend()
    ax.grid(True)

axes[-1].set_xlabel("Time")
plt.tight_layout()
plt.show()

In [ ]:
from scipy.optimize import least_squares

def _residuals(params, t, vol, sd):
    lam, k, n0 = params
    return (vol - modeleval(lam, k, n0, t)) / sd      

def fit_mle(time, vol, sd, guess=None):
    """
    time, vol, sd  : 1-D NumPy arrays
    guess          : (lam, k, n0) initial guess  (optional)
    """
    if guess is None:          # cheap, generic starting point
        lam0 = 0.2
        k0   = vol.max() * 1.5
        n00  = vol[0]
        guess = (lam0, k0, n00)

    # positivity / plausible bounds
    bounds = ([0.0,  1.0,  0.0],      # lower for (lambda, k, n0)
              [2.0,  10.0, 1.5 * vol[0]])  # upper

    sol = least_squares(_residuals, guess,
                        args=(time, vol, sd),
                        bounds=bounds,  method="trf",
                        ftol=1e-9, xtol=1e-9, gtol=1e-9)

    # Approximate covariance matrix
    # sigma^2 estimated from residual variance
    J   = sol.jac
    rss = (sol.fun**2).sum()
    dof = len(vol) - len(sol.x)
    sigma2 = rss / dof
    cov = np.linalg.inv(J.T @ J) * sigma2

    return sol.x, cov

# FIT CONTROL and IR

# --- CONTROL -------------------------------------------------------
theta_C_mle, cov_C = fit_mle(time_C, tum_vol_C, tum_sd_C,
                             guess=(0.25, 4.0, tum_vol_C[0]))
lam_C, k_C, n0_C = theta_C_mle

# --- IR ------------------------------------------------------------
theta_IR_mle, cov_IR = fit_mle(time_IR, tum_vol_IR, tum_sd_IR,
                               guess=(0.15, 3.5, tum_vol_IR[0]))
lam_IR, k_IR, n0_IR = theta_IR_mle

print("CONTROL (MLE):")
print(f"  lambda  = {lam_C:.4f}   /day")
print(f"  k  = {k_C:.4f}   $cm^3$")
print(f"  n0 = {n0_C:.4f}  $cm^3$")
print("\nIR (MLE):")
print(f"  lambda  = {lam_IR:.4f}   /day")
print(f"  k  = {k_IR:.4f}   $cm^3$")
print(f"  n0 = {n0_IR:.4f}  $cm^3$")

In [ ]:
# Metropolis–Hastings for tumour-volume model
import numpy as np
import matplotlib.pyplot as plt
import arviz as az
from tqdm.auto import tqdm
from numpy.random import default_rng


# DATA  
CONTROL = {
    "Time":   time_C,
    "Vol":    tum_vol_C,
    "SD":     tum_sd_C,
    "MLE":    (0.2822, 2.4117, 0.1160),      # lambda, k, n0 
    "prop":   (0.05,   0.5,  0.0880),     # lambda_sd, k_sd, n0_sd  (proposal s.d.)
}

IR = {
    "Time":   time_IR,
    "Vol":    tum_vol_IR,
    "SD":     tum_sd_IR,
    "MLE":    (0.2057, 1.8014, 0.2631), 
    "prop":   (0.05,   0.5,    0.1760),
}

def modeleval(lam, k, n0, t):
    """F(t) = n0·k / [ n0 + (k−n0)·e^{-lambda(t−t0)} ]"""
    t = np.asarray(t, dtype=float)
    return (n0 * k) / (n0 + (k - n0) * np.exp(-lam * (t - t[0])))


def rsseval(lam, k, n0, t, vol, sd):
    f = modeleval(lam, k, n0, t)
    return np.square(np.linalg.norm((vol - f) / sd))  


def reflect(x, lo, hi):
    if x < lo:
        return lo + (lo - x)
    if x > hi:
        return hi - (x - hi)
    return x


# 2MH SAMPLER FOR ONE CHAIN 
def mh_chain(theta0, R, t, vol, sd, n_iter, rng):
    """
    Metropolis–Hastings for lambda, k, n0.
    * theta0 : (lambda0, k0, n0)
    """
    chain = np.empty((n_iter, 3))
    chain[0] = theta0
    acc = 0

    for i in range(1, n_iter):
        prop = chain[i - 1] + rng.standard_normal(3) @ R
        lam_star, k_star, n0_star = prop

        # reflect into bounds
        lam_star = reflect(lam_star, 0.0, 0.75)
        k_star   = reflect(k_star,   1., 10.0)
        n0_star  = reflect(n0_star,  0.02, 1.0)

        rss_prev = rsseval(*chain[i - 1], t, vol, sd)
        rss_star = rsseval(lam_star, k_star, n0_star, t, vol, sd)
        alpha = np.exp(0.5 * (rss_prev - rss_star))

        if rng.random() < alpha:
            chain[i] = (lam_star, k_star, n0_star)
            acc += 1
        else:
            chain[i] = chain[i - 1]

    acc_rate = acc / (n_iter - 1)
    return chain, acc_rate

def run_group(group, n_chains=4, n_iter=50_000, seed=0):
    rng = default_rng(seed)

    lam_MLE, k_MLE, n0_MLE = group["MLE"]
    sig_lam, sig_k, sig_n0 = group["prop"]

    # proposal covariance
    Sigma = np.array([
        [sig_lam ** 2, 0.0005,       0.0005],
        [0.0005,       sig_k ** 2,   0.0005],
        [0.0005,       0.0005,       sig_n0 ** 2],
    ])
    R = np.linalg.cholesky(Sigma)

    # over-dispersed starting points around (±2 s.d.) plus the MLE itself
    starts = [
        (lam_MLE +  2*sig_lam, k_MLE +  2*sig_k, n0_MLE +  2*sig_n0),
        (lam_MLE -  2*sig_lam, k_MLE +  2*sig_k, n0_MLE -  2*sig_n0),
        (lam_MLE +  2*sig_lam, k_MLE -  2*sig_k, n0_MLE -  2*sig_n0),
        (lam_MLE,              k_MLE,            n0_MLE             ),
    ][:n_chains]

    chains = []
    acc_rates = []

    print(f"Running {n_chains} chains × {n_iter:,} iterations "
          f"({group is CONTROL and 'Control' or 'IR'} group)…")
    for s in tqdm(starts, desc="chains", leave=False):
        chain, acc = mh_chain(s, R, group["Time"], group["Vol"], group["SD"],
                              n_iter, rng)
        chains.append(chain)
        acc_rates.append(acc)

    chains = np.stack(chains)  # shape (n_chains, n_iter, 3)
    print("Acceptance rates:", ", ".join(f"{a:.3f}" for a in acc_rates))

    # TRACE PLOTS 
    param_names = ["$\lambda$", "k", "n0"]
    fig, axs = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
    for p in range(3):
        for c in range(n_chains):
            axs[p].plot(chains[c, :, p], alpha=0.6,
                        label=f"chain {c + 1}" if p == 0 else "")
        axs[p].set_ylabel(param_names[p])
    axs[-1].set_xlabel("iteration")
    axs[0].legend(ncol=n_chains, fontsize=9)
    fig.suptitle("Trace plots: " + ("Control" if group is CONTROL else "IR"))
    fig.tight_layout()
    plt.show()

    # GELMAN-RUBIN R-HAT 
    idata = az.from_dict(posterior={
        "lambda": chains[:, :, 0],
        "k":      chains[:, :, 1],
        "n0":     chains[:, :, 2],
    })
    rhat = az.rhat(idata)
    print("\nGelman–Rubin R-hat:")
    for par in ("lambda", "k", "n0"):
        print(f"  {par:>6s}: {float(rhat[par]):.4f}")

    return chains, rhat

In [ ]:
# CONTROL MH
control_chains, _ = run_group(CONTROL, n_chains=4, n_iter=50_000, seed=42)

lam_chain_C = control_chains[0, :, 0].ravel()   # lambda
k_chain_C   = control_chains[0, :, 1].ravel()   # k
n0_chain_C  = control_chains[0, :, 2].ravel()   # n0

np.savez("mh_control_chains.npz",
    lambda_chain=lam_chain_C,
    k_chain=k_chain_C,
    n0_chain=n0_chain_C)

In [ ]:
# IR MH
ir_chains, _ = run_group(IR, n_chains=4, n_iter=50_000, seed=24)

lam_chain_IR = ir_chains[0, :, 0].ravel()
k_chain_IR   = ir_chains[0, :, 1].ravel()
n0_chain_IR  = ir_chains[0, :, 2].ravel()

np.savez("mh_ir_chains.npz",
    lambda_chain=lam_chain_IR,
    k_chain=k_chain_IR,
    n0_chain=n0_chain_IR)

In [ ]:
# Load CONTROL mice
data_C = np.load("mh_control_chains.npz")
lam_chain_C    = data_C["lambda_chain"]
k_chain_C      = data_C["k_chain"]
n0_chain_C     = data_C["n0_chain"]

# Load IR mice
data_IR = np.load("mh_ir_chains.npz")
lam_chain_IR = data_IR["lambda_chain"]
k_chain_IR   = data_IR["k_chain"]
n0_chain_IR  = data_IR["n0_chain"]

Nrun      = lam_chain_C.shape[0]      # same length as MH pop size
Nrun2     = 100                   # number of mice per chain draw

tspan_fine_C  = np.arange(time_C[0], 17.0 + 1e-12, 0.01)
tspan_fine_IR = np.arange(time_IR[0], 22.0 + 1e-12, 0.01)

# time-of-death matrices and tumour sizes at death
theta_C         = np.zeros((Nrun2, Nrun))
theta_control   = np.zeros_like(theta_C)   # will hold only accepted trials
finalTumors_C   = np.zeros_like(theta_C)

theta_IR        = np.zeros((Nrun2, Nrun))
theta_treatment = np.zeros_like(theta_IR)
finalTumors_IR  = np.zeros_like(theta_IR)

# per-mouse survival curves (rows = chain index, cols = time grid)
miceSurvival_C  = np.zeros((Nrun, tspan_fine_C.size))
miceSurvival_IR = np.zeros((Nrun, tspan_fine_IR.size))

# for *accepted* mice
accepted_lambda_C  = []
accepted_k_C       = []
accepted_n0_C      = []
accepted_theta_C   = []
accepted_final_C   = []

accepted_lambda_IR = []
accepted_k_IR      = []
accepted_n0_IR     = []
accepted_theta_IR  = []
accepted_final_IR  = []

rng = np.random.default_rng(123)   # reproducible; drop seed for randomness

# MAIN ToD LOOP OVER MH POP 
for j in tqdm(range(Nrun), desc="time-of-death"):

    # CONTROL group 
    tum_C = modeleval(lam_chain_C[j], k_chain_C[j], n0_chain_C[j], tspan_fine_C)
    surv_C = sofneval(n0_chain_C[j], tum_C, SNparams_C)   # S(N)
    miceSurvival_C[j] = surv_C

    # IR group 
    tum_IR = modeleval(lam_chain_IR[j], k_chain_IR[j], n0_chain_IR[j],
                       tspan_fine_IR)
    surv_IR = sofneval(n0_chain_IR[j], tum_IR, SNparams_IR)
    miceSurvival_IR[j] = surv_IR

    # simulate Nrun2 mice for this draw 
    for h in range(Nrun2):
        coin = rng.random()
        # control 
        idx = np.argmax(surv_C <= coin) if np.any(surv_C <= coin) else None
        if idx is None:
            theta_C[h, j]      = tspan_fine_C[-1]
            finalTumors_C[h,j] = tum_C[-1]
        else:
            theta_C[h, j]      = tspan_fine_C[idx]
            finalTumors_C[h,j] = tum_C[idx]
        # IR 
        idx_IR = np.argmax(surv_IR <= coin) if np.any(surv_IR <= coin) else None
        if idx_IR is None:
            theta_IR[h, j]      = tspan_fine_IR[-1]
            finalTumors_IR[h,j] = tum_IR[-1]
        else:
            theta_IR[h, j]      = tspan_fine_IR[idx_IR]
            finalTumors_IR[h,j] = tum_IR[idx_IR]

    # accept / reject this parameter combo based on ToD window in data
    if np.median(theta_C[:, j]) < 17:
        accepted_lambda_C.append(lam_chain_C[j])
        accepted_k_C.append(k_chain_C[j])
        accepted_n0_C.append(n0_chain_C[j])
        accepted_theta_C.append(np.median(theta_C[:, j]))
        accepted_final_C.append(np.median(finalTumors_C[:, j]))
        theta_control[:, j] = theta_C[:, j]        

    if np.median(theta_IR[:, j]) < 21.5:
        accepted_lambda_IR.append(lam_chain_IR[j])
        accepted_k_IR.append(k_chain_IR[j])
        accepted_n0_IR.append(n0_chain_IR[j])
        accepted_theta_IR.append(np.median(theta_IR[:, j]))
        accepted_final_IR.append(np.median(finalTumors_IR[:, j]))
        theta_treatment[:, j] = theta_IR[:, j]

accepted_lambda_C  = np.array(accepted_lambda_C)
accepted_k_C       = np.array(accepted_k_C)
accepted_n0_C      = np.array(accepted_n0_C)
accepted_theta_C   = np.array(accepted_theta_C)
accepted_final_C   = np.array(accepted_final_C)

accepted_lambda_IR = np.array(accepted_lambda_IR)
accepted_k_IR      = np.array(accepted_k_IR)
accepted_n0_IR     = np.array(accepted_n0_IR)
accepted_theta_IR  = np.array(accepted_theta_IR)
accepted_final_IR  = np.array(accepted_final_IR)

print(f"\nAccepted Control mice : {accepted_theta_C.size}")
print(f"Accepted IR mice       : {accepted_theta_IR.size}")

In [ ]:
tspan     = np.arange(time_C[0],     time_C[-1]     + 1, 1.0)
tspan_IR  = np.arange(time_IR[0],  time_IR[-1]  + 1, 1.0)

# MODEL TRAJECTORIES FOR ACCEPTED MICE
Frun_fin      = np.empty((accepted_lambda_C.size,  tspan.size))
Frun_IR_fin   = np.empty((accepted_lambda_IR.size, tspan_IR.size))

for i, (lam, k, n0) in enumerate(
        zip(accepted_lambda_C, accepted_k_C, accepted_n0_C)):
    Frun_fin[i] = modeleval(lam, k, n0, tspan)

for i, (lam, k, n0) in enumerate(
        zip(accepted_lambda_IR, accepted_k_IR, accepted_n0_IR)):
    Frun_IR_fin[i] = modeleval(lam, k, n0, tspan_IR)

# MEAN & STANDARD DEVIATION ACROSS ACCEPTED MICE
Favg_fin      = Frun_fin.mean(axis=0)
Fstd_fin      = Frun_fin.std(axis=0)

Favg_IR_fin   = Frun_IR_fin.mean(axis=0)
Fstd_IR_fin   = Frun_IR_fin.std(axis=0)

# PLOT TUMOR VOLUME TIMECOURSE FOR ACCEPTED MICE
plt.figure(figsize=(8, 5))

# CONTROL ribbon + error bars + mean line 
plt.fill_between(tspan,
                 Favg_fin + Fstd_fin,
                 Favg_fin - Fstd_fin,
                 color='#CC5500', alpha=0.3, edgecolor="none")
plt.errorbar(time_C, tum_vol_C, yerr=tum_sd_C,
             fmt="ko", markersize=8, linewidth=2)
plt.plot(tspan, Favg_fin, color='#CC5500', linewidth=3, label="Control mean")

# IR ribbon + error bars + mean line 
plt.fill_between(tspan_IR,
                 Favg_IR_fin + Fstd_IR_fin,
                 Favg_IR_fin - Fstd_IR_fin,
                 color='#5B7C99', alpha=0.3, edgecolor="none")
plt.errorbar(time_IR, tum_vol_IR, yerr=tum_sd_IR,
             fmt="ko", markersize=8, linewidth=2)
plt.plot(tspan_IR, Favg_IR_fin, color='#5B7C99', linewidth=3, label="IR mean")

plt.xlabel("Time (days)")
plt.ylabel("Tumour volume ($cm^3$)")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Histogram plots for mean time of death
fig, axes = plt.subplots(2, 1, figsize=(8, 5), sharex=True)

# shared bin edges
hi = max(accepted_theta_C.max(), accepted_theta_IR.max())
bins = np.arange(0, np.ceil(hi) + 1)   # 1-day bins

# CONTROL 
axes[0].hist(accepted_theta_C,
             bins=bins, color='#CC5500', alpha=0.75, edgecolor="black")
axes[0].set_title("Accepted times of death: Control")
axes[0].set_ylabel("Count")

# IR 
axes[1].hist(accepted_theta_IR,
             bins=bins, color='#5B7C99', alpha=0.75, edgecolor="black")
axes[1].set_title("Accepted times of death: IR")
axes[1].set_xlabel("Time of death (days)")
axes[1].set_ylabel("Count")

plt.tight_layout()
plt.show()

In [ ]:
def plot_two(ax_row1, ax_row2, data_C, data_IR, colour_C, colour_IR, label):
    # common bin edges for visual comparison
    lo = min(data_C.min(), data_IR.min())
    hi = max(data_C.max(), data_IR.max())
    bins = np.linspace(lo, hi, 31)

    ax_row1.hist(data_C, bins=bins, color=colour_C, alpha=0.75, edgecolor="black")
    ax_row2.hist(data_IR, bins=bins, color=colour_IR, alpha=0.75, edgecolor="black")

    # identical x-limits on both subplots
    ax_row1.set_xlim(lo, hi)
    ax_row2.set_xlim(lo, hi)

    ax_row1.set_ylabel("count")
    ax_row2.set_ylabel("count")
    ax_row1.set_title(f"{label} – Control")
    ax_row2.set_title(f"{label} – IR")

# PLOT PARAMETER DISTRIBUTIONS FOR BOTH COHORTS
fig, axs = plt.subplots(2, 3, figsize=(11, 8), sharex=False)

# lambda (growth rate)
plot_two(axs[0,0], axs[1,0],
         accepted_lambda_C, accepted_lambda_IR,
         '#CC5500', '#5B7C99', "$\lambda$")

# k (carrying capacity)
plot_two(axs[0,1], axs[1,1],
         accepted_k_C, accepted_k_IR,
         '#CC5500', '#5B7C99', "K")

# n0 (initial tumour volume)
plot_two(axs[0,2], axs[1,2],
         accepted_n0_C, accepted_n0_IR,
         '#CC5500', '#5B7C99', "N0")

# theta (median ToD)
# plot_two(axs[0,3], axs[1,3],
#         accepted_theta_C, accepted_theta_IR,
#         '#CC5500', '#5B7C99', r'$\hat{\theta}$')

# shared x-labels only on the bottom row
axs[1,0].set_xlabel("parameter value")
axs[1,1].set_xlabel("parameter value")
axs[1,2].set_xlabel("parameter value")
# axs[1,3].set_xlabel("parameter value")

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df_control = pd.DataFrame({
    '$\lambda$': accepted_lambda_C,
    'K': accepted_k_C,
    'N1': accepted_n0_C
})

g = sns.pairplot(
    df_control,
    diag_kind="kde",
    kind="hist",
    corner=True,
    plot_kws={'cmap': 'Oranges'},
    diag_kws={'color': '#CC5500'}  # Burnt Orange
)

for ax in g.axes.flatten():
    if ax is not None:
        ax.set_xlabel(ax.get_xlabel(), fontsize=14, fontweight='bold')
        ax.set_ylabel(ax.get_ylabel(), fontsize=14, fontweight='bold')
        ax.tick_params(labelsize=12)

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

df_ir = pd.DataFrame({
    f'$\lambda$':  accepted_lambda_IR,
    'K':  accepted_k_IR,
    'N6': accepted_n0_IR
})

g2 = sns.pairplot(
        df_ir,
        diag_kind="kde",
        kind="hist",
        corner=True,
        plot_kws={'cmap': 'Blues'},
        diag_kws={'color': '#5B7C99'}   
)

for ax in g2.axes.flatten():
    if ax is not None:
        ax.set_xlabel(ax.get_xlabel(), fontsize=14, fontweight='bold')
        ax.set_ylabel(ax.get_ylabel(), fontsize=14, fontweight='bold')
        ax.tick_params(labelsize=12)

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ranksums
import numpy as np

control_df = pd.DataFrame({
    'Lambda': accepted_lambda_C,
    'K': accepted_k_C,
    'N(6)': accepted_n0_C,
    'N(theta)': accepted_final_C,
    'Theta': accepted_theta_C,
    'Group': 'Control'
})

treatment_df = pd.DataFrame({
    'Lambda': accepted_lambda_IR,
    'K': accepted_k_IR,
    'N(6)': accepted_n0_IR,
    'N(theta)': accepted_final_IR,
    'Theta': accepted_theta_IR,
    'Group': r'$R_x$'
})

data = pd.concat([control_df, treatment_df], ignore_index=True)

# Plotting
def get_pval_text(p_value):
    if p_value < 0.001:
        return "p-val < 0.001"
    elif p_value < 0.01:
        return "p-val < 0.01"
    elif p_value < 0.05:
        return "p-val < 0.05"
    else:
        return "p-val = ns"

parameters = [
    (r'$\lambda$', 'Lambda'), 
    (r'$K$', 'K'),             
    (r'$N(6)$', 'N(6)'),       
    (r'$N(\hat{\theta})$', 'N(theta)'),  
    (r'$\hat{\theta}$', 'Theta')  
]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
plt.subplots_adjust(hspace=0.4, wspace=0.4)

for i, (label, param_name) in enumerate(parameters):
    row, col = divmod(i, 3)
    ax = axes[row, col]
    sns.violinplot(x='Group', y=param_name, data=data, ax=ax,
                   palette={'Control': '#CC5500', r'$R_x$': '#5B7C99'})
    control_vals = data[data['Group'] == 'Control'][param_name]
    treat_vals = data[data['Group'] == r'$R_x$'][param_name]
    p_value = ranksums(control_vals, treat_vals).pvalue
    y_min = -0.08
    y_max = data[param_name].max()
    ax.set_ylim(y_min, y_max * 1.1)
    ax.text(0.73, 0.995, get_pval_text(p_value), transform=ax.transAxes,
            fontsize=12, fontweight='bold', ha='left', va='top')
    ax.set_xlabel('Group', fontsize=14, fontweight='bold')
    ax.set_ylabel(label, fontsize=14, fontweight='bold')
    ax.set_title(f'{label} Distributions', fontsize=14, fontweight='bold')

fig.delaxes(axes[1, 2])

plt.tight_layout()
plt.show()

In [ ]:
Nrun3 = len(accepted_lambda_C)   # number of accepted parameter sets
Nrun4 = 100                      # number of mice per set

tspan_fine = np.arange(time_C[0], 40.0 + 1e-12, 0.01)   # 0.01-day grid to 40 d

theta_outside_C        = np.zeros((Nrun4, Nrun3))     # time of death (rows=h, cols=j)
finalTumors_outside_C  = np.zeros_like(theta_outside_C)
miceSurvival_outside_C = np.zeros((Nrun3, tspan_fine.size))

# store who's out data
outside_params_C       = np.zeros((Nrun3, 3))         # (λ, k, n0) actually used
outside_sample_tumors  = np.zeros((Nrun3, tspan_fine.size))
median_outside_theta_C        = np.zeros(Nrun3)
median_outside_finalTumors_C  = np.zeros(Nrun3)

#SNparams_C: (rho, gamma) tuple from earlier fit

rng        = np.random.default_rng(123)  # reproducible; drop seed if not needed

# three independent permutations to shuffle indices 0..Nrun3-1
Msampler = np.vstack([rng.permutation(Nrun3) for _ in range(3)])

# WHO'S OUT DEATH ASSIGNMENT
for j in tqdm(range(Nrun3), desc="outside cohort"):

    aj, bj, cj = Msampler[:, j]   

    # ------ tumour trajectory with shuffled parameters -------------
    sample_tumour = modeleval(accepted_lambda_C[aj],
                              accepted_k_C[bj],
                              accepted_n0_C[cj],
                              tspan_fine)

    outside_sample_tumors[j] = sample_tumour
    outside_params_C[j] = (accepted_lambda_C[aj],
                           accepted_k_C[bj],
                           accepted_n0_C[cj])

    # ------ survival as a fn of tumour size ------------------------
    surv = sofneval(accepted_n0_C[j], sample_tumour, SNparams_C)
    miceSurvival_outside_C[j] = surv

    # ------ simulate Nrun4 mice for this parameter set -----
    for h in range(Nrun4):
        coin = rng.random()
        # first index where survival ≤ coin
        idx = np.argmax(surv <= coin) if np.any(surv <= coin) else None

        if idx is not None and surv[idx] <= coin:
            theta_outside_C[h, j]       = tspan_fine[idx]
            finalTumors_outside_C[h, j] = sample_tumour[idx]
        else:   # no expected death before 40 d
            theta_outside_C[h, j]       = 40.0
            finalTumors_outside_C[h, j] = sample_tumour[-1]

    # per-draw medians
    median_outside_theta_C[j]        = np.median(theta_outside_C[:, j])
    median_outside_finalTumors_C[j]  = np.median(finalTumors_outside_C[:, j])

# FILTER NON-ZERO ENTRIES
median_outside_theta_C       = median_outside_theta_C[median_outside_theta_C       > 0]
median_outside_finalTumors_C = median_outside_finalTumors_C[median_outside_finalTumors_C > 0]

outside_lambda_C = outside_params_C[:, 0][outside_params_C[:, 0] > 0]
outside_k_C      = outside_params_C[:, 1][outside_params_C[:, 1] > 0]
outside_n0_C     = outside_params_C[:, 2][outside_params_C[:, 2] > 0]

In [ ]:
# Identify external mice from window of death (16 < theta < 34)
xthetalocs = np.where((median_outside_theta_C > 16) &
                      (median_outside_theta_C < 34))[0]

# overall who's out means / stds (already computed earlier)
outside_tumorsavg = outside_sample_tumors.mean(axis=0)
outside_tumorsstd = outside_sample_tumors.std(axis=0)

# subset means / stds
subset_mean = outside_sample_tumors[xthetalocs].mean(axis=0)
subset_std  = outside_sample_tumors[xthetalocs].std(axis=0)

# Plotting...
plt.figure(figsize=(8, 6))

# ── Control ribbon + mean --------------------------------------
plt.fill_between(tspan,
                 Favg_fin - Fstd_fin,
                 Favg_fin + Fstd_fin,
                 color="blue", alpha=0.30, edgecolor="none")
plt.errorbar(time_C, tum_vol_C, yerr=tum_sd_C,
             fmt="ko", markersize=8, linewidth=2)
plt.plot(tspan, Favg_fin, color="blue", linewidth=3, label="Control mean")
plt.plot(tspan, Favg_fin + Fstd_fin, color="blue", linewidth=1)
plt.plot(tspan, Favg_fin - Fstd_fin, color="blue", linewidth=1)

# ── IR ribbon + mean -------------------------------------------
plt.fill_between(tspan_IR,
                 Favg_IR_fin - Fstd_IR_fin,
                 Favg_IR_fin + Fstd_IR_fin,
                 color="red", alpha=0.30, edgecolor="none")
plt.errorbar(time_IR, tum_vol_IR, yerr=tum_sd_IR,
             fmt="ko", markersize=8, linewidth=2)
plt.plot(tspan_IR, Favg_IR_fin, color="red", linewidth=3, label="IR mean")
plt.plot(tspan_IR, Favg_IR_fin + Fstd_IR_fin, color="red", linewidth=1)
plt.plot(tspan_IR, Favg_IR_fin - Fstd_IR_fin, color="red", linewidth=1)

# ── All who's out simulations (green) ----------------------------
plt.fill_between(tspan_fine,
                 outside_tumorsavg - outside_tumorsstd,
                 outside_tumorsavg + outside_tumorsstd,
                 color="green", alpha=0.10, edgecolor="none")
plt.plot(tspan_fine, outside_tumorsavg,
         color="green", linewidth=3, label="Outside avg")
plt.plot(tspan_fine, outside_tumorsavg + outside_tumorsstd, color="green", linewidth=1)
plt.plot(tspan_fine, outside_tumorsavg - outside_tumorsstd, color="green", linewidth=1)

# ── Who's out subset within external window (magenta) -----------------------
plt.fill_between(tspan_fine,
                 subset_mean - subset_std,
                 subset_mean + subset_std,
                 color="magenta", alpha=0.10, edgecolor="none")
plt.plot(tspan_fine, subset_mean,
         color="magenta", linewidth=3, label="Outside subset")
plt.plot(tspan_fine, subset_mean + subset_std, color="magenta", linewidth=1)
plt.plot(tspan_fine, subset_mean - subset_std, color="magenta", linewidth=1)

# ── External validation dataset --------------------------------
plt.plot(time_ext, tum_vol_ext, linewidth=3, color="black", label="External data")
plt.errorbar(time_ext, tum_vol_ext, yerr=tum_sd_ext,
             fmt="ko", markersize=8, linewidth=2)

# ---------------------------------------------------------------
plt.xlabel("Time (days)")
plt.ylabel("Tumour volume (cm³)")
plt.legend(fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
# SUBSET OF MICE WITHIN EXTERNAL WINDOW (16 < theta < 34)
xthetalocs = np.where((median_outside_theta_C > 16) &
                      (median_outside_theta_C < 34))[0]

outtums = outside_sample_tumors[xthetalocs]          

# EXTRACT TUMOR VOLS AT EXTERNAL TIMES
vals = np.array([0, 300, 600, 900, 1300, 1600, 2000])   
outtums_vals = outtums[:, vals]                         

# COUNT TUMOR VOLS WITHIN EXTERNAL ERROR BARS
ups = tum_vol_ext + tum_sd_ext
downs = tum_vol_ext - tum_sd_ext

upz   = ups - outtums_vals
downz = outtums_vals - downs
blip   = np.abs(outtums_vals - tum_vol_ext)

countu = (upz   > 0).sum(axis=1)   # # points below upper band
countd = (downz > 0).sum(axis=1)   # # points above lower band
countz = np.column_stack([countu, countd])

# “success” = all 7 points within ±1 SD of target
XYZ = (blip < tum_sd_ext).sum(axis=1) > 6      # boolean array length = #sel

# Gather parameters and tumour trajectories for mice within death window and tumor window
lamso   = outside_lambda_C[xthetalocs]
kso     = outside_k_C[xthetalocs]
n0so    = outside_n0_C[xthetalocs]
thetaso = median_outside_theta_C[xthetalocs]
wout_params = np.column_stack([lamso, kso, n0so, thetaso])

thetasout = theta_outside_C[:, xthetalocs]     # 100 × #sel

# mask of success fully recovered tumors
success_mask = XYZ

successfully_recovered_tumors = outtums[success_mask]
truncated_srts                = outtums_vals[success_mask]
srts_params                   = wout_params[success_mask]
srts_theta_full               = thetasout[:, success_mask]

# split srts_params into separate vectors & remove zeros (if any)
srts_lambda = srts_params[:, 0][srts_params[:, 0] > 0]
srts_k      = srts_params[:, 1][srts_params[:, 1] > 0]
srts_n0     = srts_params[:, 2][srts_params[:, 2] > 0]
srts_theta  = srts_params[:, 3][srts_params[:, 3] > 0]

# Kaplan–Meier curves for successfully recovered cohort
kmavg        = srts_theta_full.mean(axis=0)          # mean ToD per mouse
upperkmout   = kmavg + srts_theta_full.std(axis=0)
lowerkmout   = kmavg - srts_theta_full.std(axis=0)

srtsKMx1  = np.arange(14, 40)                       
def km_curve(arr, xgrid):
    return (len(arr) - (arr[..., None] < xgrid).sum(axis=0)) / len(arr)

srtsKMproba = km_curve(kmavg,       srtsKMx1)
srtsKMprobl = km_curve(lowerkmout,  srtsKMx1)
srtsKMprobu = km_curve(upperkmout,  srtsKMx1)

# prepend the initial (t=1, S=1) point
srtsKMx2   = np.r_[1,        srtsKMx1,   srtsKMx1]
srtsKMy2   = np.r_[1,        srtsKMproba, srtsKMproba]
srtsKMy2l  = np.r_[1,        srtsKMprobl, srtsKMprobl]
srtsKMy2u  = np.r_[1,        srtsKMprobu, srtsKMprobu]

# MATLAB’s “staircase doubling” to make a step plot
def matlab_stairs(x, y):
    """Replicates MATLAB’s manual ‘stairs’ duplication."""
    N  = len(x)
    xa = np.repeat(x, 2)[1:]         # drop first element
    ya = np.repeat(y, 2)[1:]
    return xa, ya

srtsKMx,  _ = matlab_stairs(srtsKMx2,  srtsKMy2)   # x for the main KM curve
_, srtsKMy  = matlab_stairs(srtsKMx2,  srtsKMy2)   # y main
_, srtsKMyl = matlab_stairs(srtsKMx2,  srtsKMy2l)  # lower band
_, srtsKMyu = matlab_stairs(srtsKMx2,  srtsKMy2u)  # upper band

# MATLAB then chopped first 3 points and appended 0 at the end:
srtsKMx  = srtsKMx[2:]      # discard the first 2 duplicates
srtsKMy  = srtsKMy[2:]
srtsKMyl = srtsKMyl[2:]
srtsKMyu = srtsKMyu[2:]

# Append final zero to all y-curves
#srtsKMy  = np.r_[srtsKMy,  0]
#srtsKMyl = np.r_[srtsKMyl, 0]
#srtsKMyu = np.r_[srtsKMyu, 0]

In [ ]:
def plot_two(ax_row1, ax_row2, data_C, data_IR, colour_C, colour_IR, label):
    # common bin edges for visual comparison
    lo = min(data_C.min(), data_IR.min())
    hi = max(data_C.max(), data_IR.max())
    bins = np.linspace(lo, hi, 31)

    ax_row1.hist(data_C, bins=bins, color=colour_C, alpha=0.75, edgecolor="black")
    ax_row2.hist(data_IR, bins=bins, color=colour_IR, alpha=0.75, edgecolor="black")

    # identical x-limits on both subplots
    ax_row1.set_xlim(lo, hi)
    ax_row2.set_xlim(lo, hi)

    ax_row1.set_ylabel("count")
    ax_row2.set_ylabel("count")
    ax_row1.set_title(f"{label} – Control")
    ax_row2.set_title(f"{label} – External")

# PLOT PARAMETER DISTRIBUTIONS FOR BOTH COHORTS
fig, axs = plt.subplots(2, 3, figsize=(11, 8), sharex=False)

# lambda (growth rate)
plot_two(axs[0,0], axs[1,0],
         accepted_lambda_C, srts_lambda,
         '#CC5500', '#800000', "$\lambda$")

# k (carrying capacity)
plot_two(axs[0,1], axs[1,1],
         accepted_k_C, srts_k,
         '#CC5500', '#800000', "K")

# n0 (initial tumour volume)
plot_two(axs[0,2], axs[1,2],
         accepted_n0_C, srts_n0,
         '#CC5500', '#800000', "N0")

# theta (median ToD)
# plot_two(axs[0,3], axs[1,3],
#         accepted_theta_C, accepted_theta_IR,
#         '#CC5500', '#5B7C99', r'$\hat{\theta}$')

# shared x-labels only on the bottom row
axs[1,0].set_xlabel("parameter value")
axs[1,1].set_xlabel("parameter value")
axs[1,2].set_xlabel("parameter value")
# axs[1,3].set_xlabel("parameter value")

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from scipy.spatial.distance import pdist
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from scipy.stats import entropy


# 1) Generalized variance = det(covariance matrix) ----------------------------
def generalized_variance(X):
    """Determinant of covariance matrix (0 ⇒ no spread in at least one direction)."""
    cov = np.cov(X, rowvar=False)
    return np.linalg.det(cov)


# 2) Pairwise-distance summary (mean, min, max) -------------------------------
def pairwise_distance_stats(X):
    """Euclidean distances between all points; returns mean, min, max."""
    dists = pdist(X)                       # condensed distance vector
    return dists.mean(), dists.min(), dists.max()


# 3) PCA explained variance ---------------------------------------------------
def pca_explained_variance(X, n_components=None):
    """
    Variance captured by each principal component.
    If n_components is None, it uses all components.
    Returns array of explained-variance ratios.
    """
    pca = PCA(n_components=n_components).fit(X)
    return pca.explained_variance_ratio_


# 4) Convex-hull volume (works in 2- or 3-D) ----------------------------------
def convex_hull_volume(X):
    """
    Volume (3-D) or area (2-D) of the convex hull.
    Raises QhullError if X has < d+1 non-coplanar points.
    """
    hull = ConvexHull(X)
    return hull.volume                      # area in 2-D, volume in 3-D


# 5) Shannon entropy of a multi-dimensional histogram -------------------------
def histogram_entropy(X, bins=10):
    """
    Discretise each dimension into 'bins' equal-width bins,
    count joint frequencies, compute Shannon entropy (base e).
    """
    counts, _ = np.histogramdd(X, bins=bins)
    probs = counts.ravel() / counts.sum()
    return entropy(probs, base=np.e)


# 6) Silhouette score after k-means clustering --------------------------------
def kmeans_silhouette(X, k=2):
    """
    Run k-means (k clusters) and return silhouette score
    (−1 → tightly wrong, 0 → clusters overlap, +1 → well-separated).
    """
    labels = KMeans(n_clusters=k, n_init="auto", random_state=0).fit_predict(X)
    return silhouette_score(X, labels)


X = [srts_lambda, srts_k, srts_n0]

print("Generalized variance:", generalized_variance(X))
print("Pairwise distance  (mean, min, max):", pairwise_distance_stats(X))
print("PCA variance ratios:", pca_explained_variance(X))
# print("Convex-hull volume :", convex_hull_volume(X))
print("Histogram entropy  :", histogram_entropy(X, bins=6))
print("Silhouette (k=2)   :", kmeans_silhouette(X, k=2))

In [ ]:
NDKMx = np.array([1, 18, 18, 21, 21, 23, 23, 32, 32])
NDKMy = np.array([1, 1, 0.9, 0.9, 0.7, 0.7, 0.1, 0.1, 0])

# FIGURE – Tumour volumes at external times
mean_trunc = truncated_srts.mean(axis=0)
std_trunc  = truncated_srts.std(axis=0)

plt.figure(figsize=(6, 4))
plt.fill_between(time_ext,
                 mean_trunc - std_trunc,
                 mean_trunc + std_trunc,
                 color='#7E8C54', alpha=0.10, edgecolor="none")
plt.plot(time_ext, mean_trunc,               color='#7E8C54', linewidth=2, label="Subset mean")
plt.plot(time_ext, mean_trunc + std_trunc,   color='#7E8C54', linewidth=1)
plt.plot(time_ext, mean_trunc - std_trunc,   color='#7E8C54', linewidth=1)

plt.plot(time_ext, tum_vol_ext, linewidth=3, color="black", label="Target mean")
plt.errorbar(time_ext, tum_vol_ext, yerr=tum_sd_ext,
             fmt="ko", markersize=8, linewidth=2, label="Target ± SD")

plt.xlabel("Time (days)")
plt.ylabel("Tumour volume (cm³)")
plt.legend(fontsize=9)
plt.tight_layout()
plt.show()

# FIGURE – Kaplan–Meier comparison
plt.figure(figsize=(6, 4))

# reference (NDK) curve – solid line
plt.step(NDKMx, NDKMy, where="post", linewidth=2, color="black", label="Reference KM")

# simulated KM curve
plt.step(srtsKMx, srtsKMy, where="post", linewidth=2, color='#7E8C54', label="Simulated KM")

# ± SD band around simulated KM
plt.fill_between(srtsKMx,
                 srtsKMyl, srtsKMyu,
                 color='#7E8C54', alpha=0.10, edgecolor="none",
                 label="Simulated ± SD")

plt.xlabel("Time (days)")
plt.ylabel("Survival probability")
plt.ylim(0, 1.05)
plt.legend(fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
from matplotlib.colors import LinearSegmentedColormap

tumor_data     = outside_sample_tumors[xthetalocs]
time_of_death  = median_outside_theta_C[xthetalocs]

time_ext      = np.asarray(time_ext, float)
tum_vol_ext   = np.asarray(tum_vol_ext, float)
tum_sd_ext    = np.asarray(tum_sd_ext, float)

assert tumor_data.shape[0] == time_of_death.size, \
    "Rows in tumour matrix must match length of srts_theta"

# Truncate after median death
truncated = tumor_data.copy()
n_cols    = truncated.shape[1]
cut_idx   = np.clip(np.round(time_of_death * 100).astype(int), 0, n_cols - 1)
for i, cut in enumerate(cut_idx):
    truncated[i, cut:] = np.nan

cols = np.arange(n_cols)
has_finite = np.isfinite(truncated).any(axis=0)

max_vals = np.nanmax(truncated[:, has_finite], axis=0)
min_vals = np.nanmin(truncated[:, has_finite], axis=0)
x_fit    = cols[has_finite]

# Cubic spline on columns that contain data
smax = make_interp_spline(x_fit, max_vals, k=3)
smin = make_interp_spline(x_fit, min_vals, k=3)

smooth_x   = np.linspace(x_fit.min(), x_fit.max(), 500)
smooth_max = smax(smooth_x)
smooth_min = smin(smooth_x)

# Density shading
cmap = LinearSegmentedColormap.from_list(
    'custom_moss', ['#E2E8C2', '#7E8C54', '#4B5320'], N=256
)

plt.figure(figsize=(14, 8))
for col in cols:
    col_data = truncated[:, col]
    col_data = col_data[np.isfinite(col_data)]
    if col_data.size == 0:
        continue
    density, bins = np.histogram(col_data, bins=100, density=True)
    if density.max() == 0:
        continue
    norm = density / density.max()
    bin_c = 0.5 * (bins[1:] + bins[:-1])
    for j in range(len(bin_c) - 1):
        plt.fill_betweenx(
            [bin_c[j], bin_c[j + 1]],
            col, col + 1,
            color=cmap(norm[j]),
            linewidth=0, alpha=0.7
        )

t_idx = ((time_ext - time_ext[0]) * 100).astype(int)
plt.errorbar(
    t_idx, tum_vol_ext,
    yerr=tum_sd_ext,
    fmt='o', color='black', ecolor='black',
    elinewidth=2, capsize=4, label='Validation ± SD'
)

whole_days = np.arange(1, 22)
plt.xticks((whole_days - 1) * 100, whole_days)
# plt.yticks([])
plt.xlabel('Day',  fontsize=14, fontweight='bold')
plt.ylabel('Tumour volume (cm³)', fontsize=14, fontweight='bold')
plt.title('“Who’s Out” tumour trajectories vs. validation', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()